In [1]:
# Install the required packages: Pathway for data pipelines, Bokeh for plotting, and Panel for dashboards
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
# Load the dataset from a CSV file into a pandas DataFrame
df = pd.read_csv('dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [4]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')



In [5]:
# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [6]:
# Select specific columns and save them to a new CSV file without the index
df[[
    "SystemCodeNumber", "Latitude", "Longitude",
    "Timestamp", "Occupancy", "Capacity",
    "QueueLength", "TrafficConditionNearby",
    "IsSpecialDay", "VehicleType"
]].to_csv('parking_stream.csv', index=False)

In [7]:
# Define a Pathway schema for the parking stream with column names and data types
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [8]:
# Create a replay stream from the CSV file using the defined schema and set the input rate to 1000 rows per second
stream = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=1000
)

In [9]:
# Parse Timestamp
fmt = "%Y-%m-%d %H:%M:%S"
stream_with_time = stream.with_columns(
    t = stream.Timestamp.dt.strptime(fmt),
    day = stream.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [10]:
# Map traffic conditions to numeric levels
TRAFFIC_LEVELS = {
    "low": 1.0,
    "average": 2.0,
    "high": 3.0
}

VEHICLE_WEIGHTS = {
    "car": 1.5,
    "bike": 1.0,
    "truck": 2.0,
    "cycle":0.5
}

In [11]:
import datetime
import pathway as pw

# Set pricing parameters
ALPHA = 0.1
BASE_PRICE = 10.0

# Define a daily tumbling window to aggregate occupancy and capacity,
# then calculate the average occupancy and a dynamic price for each parking system
daily_window = (
    stream_with_time
    .windowby(
        pw.this.t,  # Use the timestamp column for windowing
        instance=pw.this.SystemCodeNumber, # Group by SystemCodeNumber
        window=pw.temporal.tumbling(datetime.timedelta(days=1)), # 1-day tumbling window
        behavior=pw.temporal.exactly_once_behavior() # Ensure exactly-once semantics
    )
    .reduce(
        SystemCodeNumber = pw.this._pw_instance, # Keep SystemCodeNumber
        t = pw.this._pw_window_end, # Use window end time
        occ_sum = pw.reducers.sum(pw.this.Occupancy), # Sum of occupancy
        occ_count = pw.reducers.count(), # Count of records
        cap = pw.reducers.max(pw.this.Capacity)  # Maximum capacity in window
    )
    .with_columns(
        occ = pw.this.occ_sum / pw.this.occ_count, # Average occupancy
        price = BASE_PRICE + ALPHA * ((pw.this.occ_sum / pw.this.occ_count) / pw.this.cap)
    )
)



In [12]:
pw.io.csv.write(daily_window, 'dynamic_price_daily_per_lot.csv')
pw.run()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


Output()

In [13]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import panel as pn

# Activate Panel in notebook
pn.extension()
output_notebook()

# Load and parse data
df = pd.read_csv("dynamic_price_daily_per_lot.csv")
df["t"] = pd.to_datetime(df["t"])
df = df.sort_values(["SystemCodeNumber", "t"])

# Store individual Bokeh panes
plots = []

# Loop over each unique lot and create a figure
for lot in df["SystemCodeNumber"].unique():
    lot_df = df[df["SystemCodeNumber"] == lot]
    source = ColumnDataSource(lot_df)

    p = figure(
        height=300,
        width=900,
        x_axis_type="datetime",
        title=f"Dynamic Price for Parking Lot {lot}",
        x_axis_label="Date",
        y_axis_label="Price",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    p.line(x="t", y="price", source=source, line_width=2, color="navy", legend_label=f"Lot {lot}")
    p.circle(x="t", y="price", source=source, size=5, color="orange")

    p.add_tools(HoverTool(
        tooltips=[
            ("Lot", "@SystemCodeNumber"),
            ("Date", "@t{%F}"),
            ("Price", "@price{0.00}")
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    ))

    p.legend.visible = False  # Optional: hide legend for clarity
    plots.append(pn.pane.Bokeh(p))

# Display all graphs in a scrollable vertical layout
pn.Column(*plots).servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)

In [14]:
import pathway as pw
import datetime


# PARAMETERS

ALPHA = 0.4
BETA = 0.05
GAMMA = 0.1
DELTA = 0.1
EPSILON = 0.1
LAMBDA = 0.5
BASE_PRICE = 10.0

# INPUT STREAM
stream_mapped = (
    stream_with_time
    .with_columns(
        traffic_num = pw.if_else(
            pw.this.TrafficConditionNearby == "low", 1.0,
            pw.if_else(
                pw.this.TrafficConditionNearby == "average", 2.0,
                pw.if_else(
                    pw.this.TrafficConditionNearby == "high", 3.0,
                    2.0
                )
            )
        ),
        vehicle_weight = pw.if_else(
            pw.this.VehicleType == "car", 1.5,
            pw.if_else(
                pw.this.VehicleType == "bike", 1.0,
                pw.if_else(
                    pw.this.VehicleType == "truck", 2.0,
                    pw.if_else(
                        pw.this.VehicleType == "cycle", 0.5,
                        1.0
                    )
                )
            )
        )
    )
)

# DAILY WINDOW, group by SystemCodeNumber

demand_window = (
    stream_mapped.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        SystemCodeNumber = pw.reducers.min(pw.this.SystemCodeNumber),  # ✅ FIXED!
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(),
        cap = pw.reducers.max(pw.this.Capacity),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.traffic_num),
        is_special = pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight_sum = pw.reducers.sum(pw.this.vehicle_weight),
        row_count = pw.reducers.count()
    )
)


# Add averages
result = (
    demand_window
    .with_columns(
        occ_rate = pw.this.occ_sum / pw.this.occ_count,
        queue_avg = pw.this.queue_sum / pw.this.row_count,
        traffic_avg = pw.this.traffic_sum / pw.this.row_count,
        vehicle_weight_avg = pw.this.vehicle_weight_sum / pw.this.row_count
    )
)


#  Raw demand
result = (
    result
    .with_columns(
        demand_raw = (
            ALPHA * (pw.this.occ_rate / pw.this.cap)
            + BETA * pw.this.queue_avg
            - GAMMA * pw.this.traffic_avg
            + DELTA * pw.this.is_special
            + EPSILON * pw.this.vehicle_weight_avg
        )
    )
)

# Normalized demand
result = (
    result
    .with_columns(
        demand_norm = pw.if_else(
            pw.this.demand_raw < 0, 0.0,
            pw.if_else(
                pw.this.demand_raw > 1, 1.0,
                pw.this.demand_raw
            )
        )
    )
)


# Raw price
result = (
    result
    .with_columns(
        price_raw = BASE_PRICE * (1 + LAMBDA * pw.this.demand_norm)
    )
)


# Final bounded price
result = (
    result
    .with_columns(
        price = pw.if_else(
            pw.this.price_raw < 0.5 * BASE_PRICE, 0.5 * BASE_PRICE,
            pw.if_else(
                pw.this.price_raw > 2 * BASE_PRICE, 2 * BASE_PRICE,
                pw.this.price_raw
            )
        )
    )
)

# OUTPUT: Write with SystemCodeNumber
pw.io.csv.write(result, "baseline_model3.csv")
pw.run()


Output()

In [15]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10, Category20
import panel as pn

# Activate panel and Bokeh
pn.extension()
output_notebook()

# Load CSV and parse time
df = pd.read_csv("baseline_model3.csv")
df["t"] = pd.to_datetime(df["t"])
df = df.sort_values(["SystemCodeNumber", "t"])

# Create a list to hold individual plots
plots = []

# Loop over each lot and generate separate plots
for lot in df["SystemCodeNumber"].unique():
    lot_df = df[df["SystemCodeNumber"] == lot]
    source = ColumnDataSource(lot_df)

    p = figure(
        height=300,
        width=800,
        x_axis_type="datetime",
        title=f"Price Trend for Lot {lot}",
        x_axis_label="Date",
        y_axis_label="Price",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    p.line(x="t", y="price", source=source, line_width=2, color="navy")
    p.circle(x="t", y="price", source=source, size=5, color="orange")

    p.add_tools(HoverTool(
        tooltips=[
            ("Date", "@t{%F}"),
            ("Price", "@price{0.00}"),
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    ))

    plots.append(pn.pane.Bokeh(p))

# Display all plots in a scrollable layout
pn.Column(*plots).servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)

In [16]:
import pandas as pd
from bokeh.plotting import figure, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10
import panel as pn

# Activate Panel and Bokeh notebook output
pn.extension()
output_notebook()

# Load both CSVs
df1 = pd.read_csv("dynamic_price_daily_per_lot.csv")
df2 = pd.read_csv("baseline_model3.csv")

# Convert time columns
df1["t"] = pd.to_datetime(df1["t"])
df2["t"] = pd.to_datetime(df2["t"])

# Sort for safety
df1 = df1.sort_values(["SystemCodeNumber", "t"])
df2 = df2.sort_values(["SystemCodeNumber", "t"])

# Store plots
plots = []

#  Unique parking lots
lots = df1["SystemCodeNumber"].unique()

# Use color palette for distinction if needed
palette = Category10[10]

# Create one plot per lot
for idx, lot in enumerate(lots):
    lot_df_1 = df1[df1["SystemCodeNumber"] == lot]
    lot_df_2 = df2[df2["SystemCodeNumber"] == lot]

    # Create data sources
    source1 = ColumnDataSource(lot_df_1)
    source2 = ColumnDataSource(lot_df_2)

    # Figure
    p = figure(
        height=300,
        width=900,
        x_axis_type="datetime",
        title=f"Real-Time Dynamic Pricing vs Competitor - Lot {lot}",
        x_axis_label="Date",
        y_axis_label="Price",
        tools="pan,wheel_zoom,box_zoom,reset,save",
    )

    # Your price line
    p.line(x="t", y="price", source=source1, line_width=2, color=palette[0], legend_label="Your Price")
    p.circle(x="t", y="price", source=source1, size=5, color=palette[0])

    # Competitor price line
    p.line(x="t", y="price", source=source2, line_width=2, color=palette[1], line_dash="dashed", legend_label="Competitor Price")
    p.square(x="t", y="price", source=source2, size=5, color=palette[1])

    # Hover tool for both
    p.add_tools(HoverTool(
        renderers=[],
        tooltips=[
            ("Lot", str(lot)),
            ("Date", "@t{%F}"),
            ("Price", "@price{0.00}"),
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    ))

    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    plots.append(pn.pane.Bokeh(p))

# Display all in a scrollable dashboard
dashboard = pn.Column(*plots).servable()
dashboard


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)
    [2] Bokeh(figure)
    [3] Bokeh(figure)
    [4] Bokeh(figure)
    [5] Bokeh(figure)
    [6] Bokeh(figure)
    [7] Bokeh(figure)
    [8] Bokeh(figure)
    [9] Bokeh(figure)
    [10] Bokeh(figure)
    [11] Bokeh(figure)
    [12] Bokeh(figure)
    [13] Bokeh(figure)

In [17]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LinearAxis, Range1d
import panel as pn

# Activate Panel in notebook
pn.extension()
output_notebook()

# Load and parse data
df = pd.read_csv("dynamic_price_daily_per_lot.csv")
df["t"] = pd.to_datetime(df["t"])
df = df.sort_values(["SystemCodeNumber", "t"])

# Select 1–2 lots
selected_lots = df["SystemCodeNumber"].unique()[:2]

# Store plots
plots = []

for lot in selected_lots:
    lot_df = df[df["SystemCodeNumber"] == lot].copy()

    # Scale occupancy to price range for overlay
    price_min, price_max = lot_df["price"].min(), lot_df["price"].max()
    occ_min, occ_max = lot_df["occ"].min(), lot_df["occ"].max()

    lot_df["occ_scaled"] = (
        (lot_df["occ"] - occ_min) / (occ_max - occ_min) * (price_max - price_min) + price_min
    )

    source = ColumnDataSource(lot_df)

    p = figure(
        height=300,
        width=900,
        x_axis_type="datetime",
        title=f"Dynamic Price & Occupancy for Lot {lot}",
        x_axis_label="Date",
        y_axis_label="Price",
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )

    # Plot price line
    p.line(x="t", y="price", source=source, line_width=2, color="navy", legend_label="Price")
    p.circle(x="t", y="price", source=source, size=5, color="navy")

    # Plot scaled occupancy line
    p.line(x="t", y="occ_scaled", source=source, line_width=2, color="orange", legend_label="Occupancy (scaled)")

    # Add hover
    p.add_tools(HoverTool(
        tooltips=[
            ("Lot", "@SystemCodeNumber"),
            ("Date", "@t{%F}"),
            ("Price", "@price{0.00}"),
            ("Occupancy", "@occ{0.0}")
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    ))

    p.legend.location = "top_left"
    p.legend.click_policy = "hide"

    plots.append(pn.pane.Bokeh(p))

# Show plots in a vertical layout
pn.Column(*plots).servable()


Column
    [0] Bokeh(figure)
    [1] Bokeh(figure)